In [1]:
import numpy as np
import pandas as pd

In [2]:
import scipy.optimize as opt
import plotly.graph_objects as go
import chart_studio as cs

## Загружаем датасет

In [29]:
data_us = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [34]:
data_us.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


### Видим, что нули только в поле fips, удалим эти строки

In [35]:
data_us.isnull().sum()

date        0
county      0
state       0
fips      716
cases       0
deaths      0
dtype: int64

In [36]:
data_us = data_us.dropna()

In [37]:
counties = data_us['fips'].unique()
len(counties)

2667

## Создадим массив с данными по каждому округу

In [39]:
data_per_county = []

for ct in counties:
    county_name = data_us[data_us['fips'] == ct]['county'].iloc[0]
    work_copy = data_us[data_us['fips'] == ct][['county', 'date', 'cases', 'deaths']]
    temp = (work_copy.groupby('date').aggregate(sum))
    temp['county'] = county_name
    if len(temp) > 5:
        data_per_county.append(temp)        

In [45]:
len(data_per_county)

2517

### Посмотрим на приближаемость экспонентой в трех первых округов

In [46]:
def exp(x, A, B, C):
    return A * np.exp(B * (x - C) ** 2)

In [47]:
figs = []

for ct in data_per_county[:3]:
    dates = ct.index #array of dates
    confirmed = ct.cases #array of confirmed
    dead = ct.deaths
    county_name = ct.county[0]
    
    # Аппроксимируем аргументы функции
    popt, pcov = opt.curve_fit(exp, np.arange(confirmed.count()), confirmed,  (1e5, -1e-2, 45), maxfev=10**6)
    A, B, C = popt
    print(county_name, ': A = ', A, ', B = ', B, ' C = ', C)
    fig = go.Figure()

    fig.add_trace(go.Bar(x=dates, y=confirmed, name="Подтвержденные случаи", marker_color='orange'))

    fig.add_trace(go.Scatter(x=dates, y=exp(np.arange(confirmed.count()), A, B, C), name='Аппроксимация', marker_color='red'))
                               
    fig.add_trace(go.Bar(x=dates, y=dead, name="Умершие", marker_color='black'))

    fig.update_layout(
        title_text="Коронавирус в " + county_name,
        title_font_size=20,
        xaxis_title="Дата",
        yaxis_title="Число людей",
        barmode='overlay'
    )
    figs.append(fig)

/home/alex/Dev/Python/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning:

Covariance of the parameters could not be estimated



Snohomish : A =  2.5916477666214656e-210 , B =  2.482158144189496e-06  C =  -13972.593172639963
Cook : A =  17842.979367727676 , B =  -0.0036346500488230733  C =  86.85041042027794
Orange : A =  1304.4497787038479 , B =  -0.003918790099614135  C =  81.60144495785036


In [48]:
cs.plotly.iplot(figs[0])

In [49]:
cs.plotly.iplot(figs[1])

In [50]:
cs.plotly.iplot(figs[2])

### Видим, что не во всех округах остался экспоненциальный рост, поэтому точность предсказания будет зависеть от количества штатов, которые ушли с экспоненты

In [ ]:
arr_exps = []
i = 0
for ct in data_per_county:
    dates = ct.index #array of dates
    confirmed = ct.cases #array of confirmed
    county_name = ct.county[0]
    index = np.arange(confirmed.count())
    
    # Аппроксимируем аргументы функции
    popt, pcov = opt.curve_fit(exp, index, confirmed,  (1e5, -1e-2, 45), maxfev=10**6)
    A, B, C = popt
    
    # Выводим аргументы для функции роста числа зараженных в данном округе
    print(i, county_name, ': A = ', A, ', B = ', B, ' C = ', C)
    
    # Так как наша аппроксимирую
    arr_exps.append(max(exp(index[-1] + 4, A, B, C), confirmed[-1]))
    print('Состояние на 12.04: ', confirmed[-1])
    print('Предсказание на 16.04: ', arr_exps[-1], '\n')
    i += 1

0 Snohomish : A =  2.5916477666214656e-210 , B =  2.482158144189496e-06  C =  -13972.593172639963
Состояние на 12.04:  1866
Предсказание на 16.04:  2960.7350181583834 

1 Cook : A =  17842.979367727676 , B =  -0.0036346500488230733  C =  86.85041042027794
Состояние на 12.04:  14585
Предсказание на 16.04:  16906.937333600006 

2 Orange : A =  1304.4497787038479 , B =  -0.003918790099614135  C =  81.60144495785036
Состояние на 12.04:  1277
Предсказание на 16.04:  1303.6380314699722 

3 Maricopa : A =  1.5469563287179022e-219 , B =  5.165572802365809e-06  C =  -9874.393709211945
Состояние на 12.04:  1960
Предсказание на 16.04:  3392.666304854802 

4 Los Angeles : A =  1.3051216755318733 , B =  -0.03624348736991086  C =  26.9523533859352
Состояние на 12.04:  9192
Предсказание на 16.04:  9192 

5 Santa Clara : A =  1631.2620298388495 , B =  -0.002783235717117351  C =  75.39704338794513
Состояние на 12.04:  1621
Предсказание на 16.04:  1629.6122494545612 

6 Suffolk : A =  7301.283896865491 

61 Chatham : A =  5.555262463859092e+99 , B =  -1.3427159899175742e-05  C =  4138.280840409605
Состояние на 12.04:  63
Предсказание на 16.04:  70.7674773963712 

62 Tulsa : A =  325.7330507771459 , B =  -0.009224110820517058  C =  36.23940091119909
Состояние на 12.04:  332
Предсказание на 16.04:  332 

63 Delaware : A =  3614.0747127828326 , B =  -0.0029180546257953194  C =  53.49135951419371
Состояние на 12.04:  1594
Предсказание на 16.04:  2292.218090420595 

64 Wayne : A =  69.95928210525956 , B =  -0.006342551748014088  C =  41.364380541560685
Состояние на 12.04:  63
Предсказание на 16.04:  69.90039274097313 

65 Charleston : A =  331.7171697638699 , B =  -0.0074126628754391105  C =  34.84758329934469
Состояние на 12.04:  339
Предсказание на 16.04:  339 

66 Kershaw : A =  181.99385161392988 , B =  -0.002794385879459485  C =  38.77964171190079
Состояние на 12.04:  179
Предсказание на 16.04:  179.5038411980422 

67 Davis : A =  196.14629531188652 , B =  -0.004340625033878929  C =  3

138 Charlton : A =  6.896473697028284e+86 , B =  -2.3342104459551373e-06  C =  9267.215744712963
Состояние на 12.04:  3
Предсказание на 16.04:  3 

139 Kane : A =  507.6632824774093 , B =  -0.0034094193319832706  C =  41.60411320596177
Состояние на 12.04:  403
Предсказание на 16.04:  472.2677113072045 

140 McHenry : A =  283.43681257783845 , B =  -0.004193642831508586  C =  39.62888464104352
Состояние на 12.04:  235
Предсказание на 16.04:  275.3400295338385 

141 Adams : A =  9.162229055373898e+159 , B =  -3.7874611596864452e-06  C =  9877.122032076628
Состояние на 12.04:  5
Предсказание на 16.04:  5 

142 Boone : A =  116.76744056866679 , B =  -0.006734907388463151  C =  36.628483718283654
Состояние на 12.04:  109
Предсказание на 16.04:  116.65894618632143 

143 Orleans : A =  5609.822148959908 , B =  -0.006577640616171029  C =  32.90473333564025
Состояние на 12.04:  5600
Предсказание на 16.04:  5600 

144 Essex : A =  4700.11245572959 , B =  -0.003677028955797151  C =  43.5388945450

193 Bon Homme : A =  2.781395336884216e+152 , B =  -2.834781331977207e-06  C =  11141.815000982377
Состояние на 12.04:  4
Предсказание на 16.04:  4 

194 Gregg : A =  39.9913476242318 , B =  -0.018992967842857627  C =  31.45172211337579
Состояние на 12.04:  39
Предсказание на 16.04:  39 

195 Chittenden : A =  359.9791914325537 , B =  -0.005448539700641149  C =  32.67033336400372
Состояние на 12.04:  364
Предсказание на 16.04:  364 

196 Arlington : A =  1594.5955502242055 , B =  -0.0014485939974430342  C =  63.64268532716273
Состояние на 12.04:  366
Предсказание на 16.04:  527.1465460764028 

197 Island : A =  154.43784148902512 , B =  -0.004719124396552918  C =  29.481603284926
Состояние на 12.04:  158
Предсказание на 16.04:  158 

198 Thurston : A =  81.08994141623846 , B =  -0.004967365880694808  C =  30.868505027558207
Состояние на 12.04:  81
Предсказание на 16.04:  81 

199 Fond du Lac : A =  649.7127762872591 , B =  -0.00036807028523522614  C =  113.21278914417908
Состояние на 1

/home/alex/Dev/Python/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

overflow encountered in exp

/home/alex/Dev/Python/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

overflow encountered in multiply



...

679 Pulaski : A =  2.4375873448470305e+67 , B =  -2.523176940661309e-05  C =  2475.2503816369817
Состояние на 12.04:  31
Предсказание на 16.04:  57.413201659513625 

680 Scott : A =  23.99889710641102 , B =  -0.008862236686874449  C =  24.053500905253838
Состояние на 12.04:  26
Предсказание на 16.04:  26 

681 Assumption : A =  114.30533262855174 , B =  -0.010590660750713788  C =  24.754513216937504
Состояние на 12.04:  116
Предсказание на 16.04:  116 

682 Calcasieu : A =  260.1434742827876 , B =  -0.007104202904897153  C =  26.041809160162288
Состояние на 12.04:  259
Предсказание на 16.04:  259 

683 Iberia : A =  132.88154042907829 , B =  -0.007415339393316187  C =  25.655106935574462
Состояние на 12.04:  136
Предсказание на 16.04:  136 

684 Iberville : A =  230.9164329131952 , B =  -0.009024316124882092  C =  25.63408855845619
Состояние на 12.04:  232
Предсказание на 16.04:  232 

685 Lafayette : A =  330.9237414743299 , B =  -0.008841840468060667  C =  22.431210178206882
Состояни

746 Tucker : A =  2.8460071584734667e+65 , B =  -1.7684292574448357e-06  C =  9216.255817805193
Состояние на 12.04:  4
Предсказание на 16.04:  4.120592223910047 

747 Bayfield : A =  3.1068288913932287 , B =  -0.00411731685598626  C =  20.016506869173742
Состояние на 12.04:  3
Предсказание на 16.04:  3 

748 Calumet : A =  4.254062534148383 , B =  -0.004702057486276189  C =  24.436029497056698
Состояние на 12.04:  4
Предсказание на 16.04:  4.00742712142675 

749 Columbia : A =  25.55203473875445 , B =  -0.004140257385264322  C =  25.322634850862396
Состояние на 12.04:  24
Предсказание на 16.04:  24.804830430935752 

750 Eau Claire : A =  21.51089514080478 , B =  -0.005889976250764961  C =  21.875804427956485
Состояние на 12.04:  21
Предсказание на 16.04:  21 

751 Ozaukee : A =  -180421.20818952724 , B =  -0.03343874898534647  C =  59.2315044592413
Состояние на 12.04:  72
Предсказание на 16.04:  72 

752 Sauk : A =  24.68246565243105 , B =  -0.0037207234925699414  C =  25.4658341020666

806 Clay : A =  1.5998554375900094e+47 , B =  -5.217419727359162e-05  C =  1442.0289864491392
Состояние на 12.04:  40
Предсказание на 16.04:  68.21404819340582 

807 Fillmore : A =  10.349243765908087 , B =  -0.005574415017519906  C =  19.15072380438848
Состояние на 12.04:  10
Предсказание на 16.04:  10 

808 Adams : A =  9.991934205257782e+61 , B =  -4.26701953413779e-05  C =  1826.56708800358
Состояние на 12.04:  51
Предсказание на 16.04:  96.98232444880655 

809 Franklin : A =  7.939984649191852e+155 , B =  -5.142817327376662e-06  C =  8351.451979654124
Состояние на 12.04:  10
Предсказание на 16.04:  13.374580299643904 

810 Humphreys : A =  159330.31745708198 , B =  -0.00028907829399258094  C =  206.33088510401774
Состояние на 12.04:  9
Предсказание на 16.04:  14.615786590778336 

811 Lafayette : A =  29.46711904506005 , B =  -0.0033020040495596215  C =  25.3346976754973
Состояние на 12.04:  29
Предсказание на 16.04:  29.19851378853373 

812 Lawrence : A =  -147529.30699281354 , B 

873 Brown : A =  2.913970169126728e+160 , B =  -5.6743946878997965e-06  C =  8067.372925612882
Состояние на 12.04:  10
Предсказание на 16.04:  14.113779351400314 

874 Cass : A =  5.120377959075247 , B =  -0.005752129095684704  C =  21.462011795802677
Состояние на 12.04:  5
Предсказание на 16.04:  5 

875 Fannin : A =  2.2803399756874746e+160 , B =  -9.490329239902538e-06  C =  6246.765812294691
Состояние на 12.04:  5
Предсказание на 16.04:  8.165624975652852 

876 Hockley : A =  16.794522630795605 , B =  -0.0017154063276912043  C =  34.62512455014765
Состояние на 12.04:  14
Предсказание на 16.04:  15.200293624921041 

877 Upshur : A =  3.788831497450482e+88 , B =  -1.5639761015335137e-05  C =  3616.2509231917034
Состояние на 12.04:  8
Предсказание на 16.04:  11.903703500243276 

878 Addison : A =  55.37807910270525 , B =  -0.004951396306442974  C =  24.545006753959726
Состояние на 12.04:  55
Предсказание на 16.04:  55 

879 Rutland : A =  35.72137990976689 , B =  -0.009860939433906481

931 Spencer : A =  1.2615542555310003e+45 , B =  -2.048872368329919e-05  C =  2254.2600912109315
Состояние на 12.04:  6
Предсказание на 16.04:  8.324725049708277 

932 Avoyelles : A =  60.82385978270057 , B =  -0.006070035188232652  C =  23.337006665580443
Состояние на 12.04:  61
Предсказание на 16.04:  61 

933 Beauregard : A =  -41577.55222267758 , B =  -0.01665784620116741  C =  65.07788641111766
Состояние на 12.04:  27
Предсказание на 16.04:  27 

934 Bienville : A =  129.98463667913606 , B =  -0.0028668754775642074  C =  43.70756313512292
Состояние на 12.04:  33
Предсказание на 16.04:  52.904326339102916 

935 Claiborne : A =  41.32359302370993 , B =  -0.011588663752155016  C =  21.95127838547952
Состояние на 12.04:  41
Предсказание на 16.04:  41 

936 Evangeline : A =  38.99130857368044 , B =  -0.005791141076560288  C =  25.629619607789376
Состояние на 12.04:  36
Предсказание на 16.04:  38.96034468971655 

937 Ouachita : A =  367.2503882883088 , B =  -0.01007885394887997  C =  23

In [24]:
exps = np.array(arr_exps)
round(exps.sum())

626354.0

## Итог:

Всего на 16.04 имеем аппроксимацию в 626354 зараженных